## Jリーグ観客数予測

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [7]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submit.csv', header=None)

In [3]:
train.head()

,id,y,year,stage,match,gameday,time,home,away,stadium,tv
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋
2,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪
3,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島
4,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1721 entries, 0 to 1720
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       1721 non-null   int64 
 1   y        1721 non-null   int64 
 2   year     1721 non-null   int64 
 3   stage    1721 non-null   object
 4   match    1721 non-null   object
 5   gameday  1721 non-null   object
 6   time     1721 non-null   object
 7   home     1721 non-null   object
 8   away     1721 non-null   object
 9   stadium  1721 non-null   object
 10  tv       1721 non-null   object
dtypes: int64(3), object(8)
memory usage: 148.0+ KB


In [5]:
test.head()

,id,year,stage,match,gameday,time,home,away,stadium,tv
0,15822,2014,Ｊ１,第１８節第１日,08/02(土),19:04,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス
1,15823,2014,Ｊ１,第１８節第１日,08/02(土),18:34,鹿島アントラーズ,サンフレッチェ広島,県立カシマサッカースタジアム,スカパー！／スカパー！プレミアムサービス
2,15824,2014,Ｊ１,第１８節第１日,08/02(土),19:04,浦和レッズ,ヴィッセル神戸,埼玉スタジアム２００２,スカパー！／スカパー！プレミアムサービス／ＮＨＫ ＢＳ１／テレ玉
3,15825,2014,Ｊ１,第１８節第１日,08/02(土),19:03,柏レイソル,川崎フロンターレ,日立柏サッカー場,スカパー！／スカパー！プレミアムサービス
4,15827,2014,Ｊ１,第１８節第１日,08/02(土),19:03,アルビレックス新潟,セレッソ大阪,デンカビッグスワンスタジアム,スカパー！／スカパー！プレミアムサービス


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       313 non-null    int64 
 1   year     313 non-null    int64 
 2   stage    313 non-null    object
 3   match    313 non-null    object
 4   gameday  313 non-null    object
 5   time     313 non-null    object
 6   home     313 non-null    object
 7   away     313 non-null    object
 8   stadium  313 non-null    object
 9   tv       313 non-null    object
dtypes: int64(2), object(8)
memory usage: 24.6+ KB


In [8]:
sample.head(5)

,0,1
0,15822,10662.395119
1,15823,10662.395119
2,15824,10662.395119
3,15825,10662.395119
4,15827,10662.395119


In [9]:
y = train['y']

In [10]:
y

0       18250
1       24316
2       17066
3       29603
4       25353
        ...  
1716     3756
1717     7842
1718     5212
1719     3883
1720     5028
Name: y, Length: 1721, dtype: int64

In [12]:
train = train.drop('y', axis=1)

In [13]:
train.head(0)

,id,year,stage,match,gameday,time,home,away,stadium,tv


In [14]:
train.describe()

,id,year
count,1721.000000,1721.000000
mean,15045.691458,2012.815224
std,648.205749,0.757613
min,13994.000000,2012.000000
25%,14474.000000,2012.000000
50%,15044.000000,2013.000000
75%,15528.000000,2013.000000
max,16237.000000,2014.000000


In [32]:
train['stage'][train['stage'].str.contains('１') == True] = 1
train['stage'][train['stage'].str.contains('２') == True] = 2

test['stage'][test['stage'].str.contains('１') == True] = 1
test['stage'][test['stage'].str.contains('２') == True] = 2

<ipython-input-32-ad1c3f97fc13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['stage'][train['stage'].str.contains('１') == True] = 1
<ipython-input-32-ad1c3f97fc13>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['stage'][train['stage'].str.contains('２') == True] = 2
<ipython-input-32-ad1c3f97fc13>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['stage'][test['stage'].str.contains('１') == True] = 1
<ipython-input-

In [27]:
train['stage']

0       1
1       1
2       1
3       1
4       1
       ..
1716    2
1717    2
1718    2
1719    2
1720    2
Name: stage, Length: 1721, dtype: object

In [26]:
train['stage'].value_counts()

2    1046
1     675
Name: stage, dtype: int64

In [33]:
test['stage'].value_counts()

2    174
1    139
Name: stage, dtype: int64

In [34]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       313 non-null    int64 
 1   year     313 non-null    int64 
 2   stage    313 non-null    object
 3   match    313 non-null    object
 4   gameday  313 non-null    object
 5   time     313 non-null    object
 6   home     313 non-null    object
 7   away     313 non-null    object
 8   stadium  313 non-null    object
 9   tv       313 non-null    object
dtypes: int64(2), object(8)
memory usage: 24.6+ KB


In [35]:
train.head(3)

,id,year,stage,match,gameday,time,home,away,stadium,tv
0,13994,2012,1,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合
1,13995,2012,1,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋
2,13996,2012,1,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪


In [53]:
buf1 = train['gameday'].apply(lambda x: x.split('(')[1])
week = buf1.apply(lambda x: x.split(')')[0])
buf1 = train['gameday'].apply(lambda x: x.split('/')[0])
month = buf1.astype(int)
buf1 = train['gameday'].apply(lambda x: x.split('/')[1])
day = buf1.apply(lambda x: x.split('(')[0]).astype(int)

train = train.drop('gameday', axis=1)
train['month'] = month
train['day'] = day
train['week'] = week

In [54]:
train.head()

,id,year,stage,match,time,home,away,stadium,tv,month,day,week
0,13994,2012,1,第１節第１日,14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合,3,10,土
1,13995,2012,1,第１節第１日,14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋,3,10,土
2,13996,2012,1,第１節第１日,14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪,3,10,土
3,13997,2012,1,第１節第１日,14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島,3,10,土
4,13998,2012,1,第１節第１日,14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌,3,10,土


In [55]:
buf1 = test['gameday'].apply(lambda x: x.split('(')[1])
week = buf1.apply(lambda x: x.split(')')[0])
buf1 = test['gameday'].apply(lambda x: x.split('/')[0])
month = buf1.astype(int)
buf1 = test['gameday'].apply(lambda x: x.split('/')[1])
day = buf1.apply(lambda x: x.split('(')[0]).astype(int)

test = test.drop('gameday', axis=1)
test['month'] = month
test['day'] = day
test['week'] = week

In [56]:
test.head()

,id,year,stage,match,time,home,away,stadium,tv,month,day,week
0,15822,2014,1,第１８節第１日,19:04,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス,8,2,土
1,15823,2014,1,第１８節第１日,18:34,鹿島アントラーズ,サンフレッチェ広島,県立カシマサッカースタジアム,スカパー！／スカパー！プレミアムサービス,8,2,土
2,15824,2014,1,第１８節第１日,19:04,浦和レッズ,ヴィッセル神戸,埼玉スタジアム２００２,スカパー！／スカパー！プレミアムサービス／ＮＨＫ ＢＳ１／テレ玉,8,2,土
3,15825,2014,1,第１８節第１日,19:03,柏レイソル,川崎フロンターレ,日立柏サッカー場,スカパー！／スカパー！プレミアムサービス,8,2,土
4,15827,2014,1,第１８節第１日,19:03,アルビレックス新潟,セレッソ大阪,デンカビッグスワンスタジアム,スカパー！／スカパー！プレミアムサービス,8,2,土


In [81]:
def get_match(x):
    b = x[1:3]
    if b[1] == "節":
        b = b[0]
    return int(b)

In [83]:
train['match'] = train['match'].apply(lambda x: get_match(x))

In [84]:
test['match'] = test['match'].apply(lambda x: get_match(x))

In [134]:
train.head(50)

,id,year,stage,match,time,home,away,stadium,tv,month,day,week
0,13994,2012,1,1,14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合,3,10,土
1,13995,2012,1,1,14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋,3,10,土
2,13996,2012,1,1,14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪,3,10,土
3,13997,2012,1,1,14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島,3,10,土
4,13998,2012,1,1,14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌,3,10,土
5,13999,2012,1,1,14:05,サガン鳥栖,セレッソ大阪,ベストアメニティスタジアム,スカパー／ｅ２／スカパー光／ＮＨＫ佐賀,3,10,土
6,14000,2012,1,1,17:05,川崎フロンターレ,アルビレックス新潟,等々力陸上競技場,スカパー／ｅ２／スカパー光,3,10,土
7,14001,2012,1,1,19:04,大宮アルディージャ,ＦＣ東京,ＮＡＣＫ５スタジアム大宮,スカパー／ｅ２／スカパー光／ＮＨＫ ＢＳ１,3,10,土
8,14002,2012,1,1,14:04,柏レイソル,横浜Ｆ・マリノス,日立柏サッカー場,スカパー／ｅ２／スカパー光（ＴＢＳチャンネル）,3,11,日
9,14004,2012,1,2,14:04,浦和レッズ,柏レイソル,埼玉スタジアム２００２,スカパー／ｅ２／スカパー光／ＮＨＫ総合,3,17,土


In [156]:
def Is_tijouha(a):
    for x in a:
        if "スカパー" in x:
            continue
        elif "２" in x:
            continue
        else:
            return 1
    
    return 0
    

In [158]:
train['tijouha'] = train['tv'].apply(lambda x: Is_tijouha(x.split('／')))
train.head(50)

,id,year,stage,match,time,home,away,stadium,tv,month,day,week,tijouha
0,13994,2012,1,1,14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合,3,10,土,1
1,13995,2012,1,1,14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋,3,10,土,1
2,13996,2012,1,1,14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪,3,10,土,1
3,13997,2012,1,1,14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島,3,10,土,1
4,13998,2012,1,1,14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌,3,10,土,1
5,13999,2012,1,1,14:05,サガン鳥栖,セレッソ大阪,ベストアメニティスタジアム,スカパー／ｅ２／スカパー光／ＮＨＫ佐賀,3,10,土,1
6,14000,2012,1,1,17:05,川崎フロンターレ,アルビレックス新潟,等々力陸上競技場,スカパー／ｅ２／スカパー光,3,10,土,0
7,14001,2012,1,1,19:04,大宮アルディージャ,ＦＣ東京,ＮＡＣＫ５スタジアム大宮,スカパー／ｅ２／スカパー光／ＮＨＫ ＢＳ１,3,10,土,1
8,14002,2012,1,1,14:04,柏レイソル,横浜Ｆ・マリノス,日立柏サッカー場,スカパー／ｅ２／スカパー光（ＴＢＳチャンネル）,3,11,日,0
9,14004,2012,1,2,14:04,浦和レッズ,柏レイソル,埼玉スタジアム２００２,スカパー／ｅ２／スカパー光／ＮＨＫ総合,3,17,土,1


In [159]:
test['tijouha'] = test['tv'].apply(lambda x: Is_tijouha(x.split('／')))
test.head(50)

,id,year,stage,match,time,home,away,stadium,tv,month,day,week,tijouha
0,15822,2014,1,18,19:04,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス,8,2,土,0
1,15823,2014,1,18,18:34,鹿島アントラーズ,サンフレッチェ広島,県立カシマサッカースタジアム,スカパー！／スカパー！プレミアムサービス,8,2,土,0
2,15824,2014,1,18,19:04,浦和レッズ,ヴィッセル神戸,埼玉スタジアム２００２,スカパー！／スカパー！プレミアムサービス／ＮＨＫ ＢＳ１／テレ玉,8,2,土,1
3,15825,2014,1,18,19:03,柏レイソル,川崎フロンターレ,日立柏サッカー場,スカパー！／スカパー！プレミアムサービス,8,2,土,0
4,15827,2014,1,18,19:03,アルビレックス新潟,セレッソ大阪,デンカビッグスワンスタジアム,スカパー！／スカパー！プレミアムサービス,8,2,土,0
5,15828,2014,1,18,18:03,ガンバ大阪,横浜Ｆ・マリノス,万博記念競技場,スカパー！／スカパー！プレミアムサービス,8,2,土,0
6,15829,2014,1,18,18:34,徳島ヴォルティス,ヴァンフォーレ甲府,鳴門・大塚スポーツパーク ポカリスエットスタジアム,スカパー！／スカパー！プレミアムサービス,8,2,土,0
7,15830,2014,1,18,19:04,サガン鳥栖,名古屋グランパス,ベストアメニティスタジアム,スカパー！／スカパー！プレミアムサービス,8,2,土,0
8,15831,2014,1,19,19:04,大宮アルディージャ,ガンバ大阪,ＮＡＣＫ５スタジアム大宮,スカパー！／スカパー！プレミアムサービス／テレ玉,8,9,土,1
9,15832,2014,1,19,19:04,川崎フロンターレ,浦和レッズ,等々力陸上競技場,スカパー！／スカパー！プレミアムサービス,8,9,土,0


In [ ]:
tr